In [3]:
import numpy as np
import os

In [1]:
root_path = '/dfs/scratch0/vschen/mri-data'

In [4]:
vocab_matrix = np.load(os.path.join(root_path, 'relative_train_vocab_matrix.npy'))

In [9]:
gt_labels = np.load(os.path.join(root_path, 'train_labels.npy'))

In [10]:
vocab_matrix.shape

(10, 636)

In [11]:
gt_labels.shape

(636,)

In [13]:
import sys
sys.path.append('/dfs/scratch0/paroma/numbskull/') 
sys.path.append('/dfs/scratch0/paroma/coral/')

In [14]:
class PrimitiveObject(object):

    def save_primitive_matrix(self,primitive_mtx):
        self.primitive_mtx = primitive_mtx
        self.discrete_primitive_mtx = primitive_mtx
        self.num_primitives = np.shape(self.primitive_mtx)[1]
    
    def save_primitive_names(self,names):
        self.primitive_names = names
        if len(self.primitive_names) != self.num_primitives:
            Exception('Incorrect number of Primitive Names')
            
def create_primitives(vocab_matrix):
    m = 5
    num_examples = vocab_matrix.shape[1]
    primitive_mtx = np.zeros((num_examples, m))
    for i in range(num_examples):
        primitive_mtx[i, 0] = vocab_matrix[0, :][i] # area
        primitive_mtx[i, 1] = vocab_matrix[1, :][i] # eccentricity
        primitive_mtx[i, 2] = vocab_matrix[6, :][i] # perimeter
        primitive_mtx[i, 3] = vocab_matrix[8, :][i] # intensity
    
    
    primitive_mtx[:, 4] = primitive_mtx[:, 0]/(primitive_mtx[:, 2]**2.) # ratio
    P = PrimitiveObject()
    P.save_primitive_matrix(primitive_mtx)
    return P

def create_primitives_bsa(vocab_matrix, normal_matrix):
    m = 5
    num_examples = vocab_matrix.shape[1]
    primitive_mtx = np.zeros((num_examples, m))
    for i in range(num_examples):
        primitive_mtx[i, 0] = vocab_matrix[0, :][i] # area
        primitive_mtx[i, 1] = vocab_matrix[1, :][i] # eccentricity
        primitive_mtx[i, 2] = vocab_matrix[6, :][i] # perimeter
        primitive_mtx[i, 3] = vocab_matrix[8, :][i] # intensity
    
    
    primitive_mtx[:, 4] = normal_matrix[0,:]/(normal_matrix[6,:]**2.) # ratio
    P = PrimitiveObject()
    P.save_primitive_matrix(primitive_mtx)
    return P

In [15]:
P_train = create_primitives(vocab_matrix)

primitive_names = ['area', 'eccentricity', 'perimeter', 'intensity', 'ratio']

In [16]:
def lf_area(area):
    if area >= 2.13:
        return -1 
    if area <= 0.99: 
        return 1
    return 0

def lf_eccentricity(eccentricity):
    if eccentricity >= 0.011: 
        return 1 
    if eccentricity <= 0.010:
        return -1  
    return 0
        
def lf_perimeter(perimeter):
    if perimeter <= 0.49: 
        return 1 
    return 0
    
def lf_intensity(intensity):
    if intensity >= 2.65: 
        return 1
    if intensity <= 2.0: 
        return -1
    return 0

def lf_ratio(ratio):
    if ratio >= 4.3: 
        return -1
    if ratio <= 4.15:
        return 1
    return 0

In [23]:
from coral.static_analysis.dependency_learning import find_dependencies
L_names = [lf_area, lf_eccentricity, lf_perimeter, lf_intensity, lf_ratio]
L_deps = find_dependencies(L_names, primitive_names)

In [24]:
num_examples_train = P_train.primitive_mtx.shape[0]
L = np.zeros((len(L_names), num_examples_train))
for i in range(num_examples_train):
    for j in range(5):
        vocab_elems = P_train.primitive_mtx[i,L_deps[j]]
        L[j,i] = L_names[j](*vocab_elems)
unlabeled = np.sum(np.abs(L), axis=0)

In [27]:
L.T

array([[ 1.,  1.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  1.],
       [ 1.,  1.,  1.,  0.,  1.],
       ...,
       [ 0.,  1.,  0., -1.,  1.],
       [ 0.,  1.,  0.,  0.,  1.],
       [ 1.,  1.,  0., -1.,  1.]])

In [28]:
np.save('L_dev.npy', L)
np.save('y_dev.npy', gt_labels)